In [30]:
print("ok")

ok


In [31]:
%pwd

'c:\\Users\\Parsh\\OneDrive\\Desktop\\project_final\\EDUBOT\\research'

In [32]:
import os
os.chdir("../")

In [33]:
%pwd

'c:\\Users\\Parsh\\OneDrive\\Desktop\\project_final\\EDUBOT'

In [34]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [35]:
#extract data from the pdf file
def load_pdf_file(Data):
    loader = DirectoryLoader(Data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [36]:
extracted_data = load_pdf_file(Data='Data/')

In [37]:
extracted_data

[Document(metadata={'source': 'Data\\book.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': 'Data\\book.pdf', 'page': 1}, page_content='“Several\tgenerations\tof\tcounselors\tand\tphysicians\thave\thad\tthe\tbenefit\tof\nEdmund\tBourne’s\tintensely\tresearched\tand\textremely\thandy\t\nThe\tAnxiety\tand\nPhobia\tWorkbook\n.\tI\tknow\tfew\tcolleagues\twho\tdo\tnot\thave\ta\tcopy\ton\ttheir\nbookshelves\tthat\tthey\tconsult\tregularly\twhen\ttreating\tpatients\tsuffering\twith\nanxiety-related\tdisorders.\tThis\tbook\tis\tespecially\timportant\ttoday\tas\tclimate\nchange\tcontinues\tto\tinflame\tthe\tanxieties\tof\tour\talready-stressed\tsocial\tfabric.\nThe\tWHO\tand\tNIH\tare\tall\treporting\tsweeping\tincreases\tin\t‘ecoanxiety.’\tThe\nrecent\tAPA\tspecial\treport,\t\nMental\tHealth\tand\tOur\tChanging\tClimate\n,\tfound\tthat\nUS-based\thurricanes\tmore\tthan\tdoubled\trates\tof\tsuicide\tand\tsuicidal\tideation,\nwith\tnearly\t50\tpercent\tof\tthe\tcatchment\tgroup\t

In [38]:
# split the data into text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [39]:
text_chunks = text_split(extracted_data)
print("length of text chunks",len(text_chunks))

length of text chunks 3077


In [1]:
# text_chunks

In [40]:
pip install langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

In [6]:
# download the embedding from hugging face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [7]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [8]:
embeddings = download_hugging_face_embeddings()

c:\Users\Parsh\anaconda3\envs\edubot\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Parsh\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [9]:
query_result = embeddings.embed_query("Hello world")
print("length", len(query_result))

length 384


In [11]:
# query_result

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
import os

In [92]:
PINECONE_API_KEY = os.environ.get('972d5b82-e75a-43ba-b708-9f6622ccbbe')
GROQ_API_KEY = os.environ.get("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")

In [69]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="972d5b82-e75a-43ba-b708-9f6622ccbbe1")

index_name = "edubot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [71]:
import os
os.environ["PINECONE_API_KEY"]="972d5b82-e75a-43ba-b708-9f6622ccbbe1"
os.environ["GROQ_API_KEY"]="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"

In [74]:
# embed each chunk and upsert the embeddings into your pinecone index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding=embeddings,
)

In [75]:
#load existing index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding=embeddings,
)


In [76]:
docsearch

In [77]:
retriver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [78]:
retrived_docs = retriver.invoke("what is stress?")

In [79]:
retrived_docs

[Document(id='265e225c-a526-4748-9a25-ef70906048c4', metadata={'page': 335.0, 'source': 'Data\\book.pdf'}, page_content='Temper\tflare-ups\nStomach\tpain\tor\tulcer\nCrying\tspells\nConstipation\nNightmares\nHypoglycemia\nApathy\nAppetite\tchange\nSexual\tproblems\nColds\nWeight\tchange\nProfuse\tperspiration\nOvereating\nHeart\tbeats\trapidly\tor\tpounds,\teven\tat\trest\nWhen\tnervous,\tuse\tof\talcohol,\tcigarettes,\t\nor\trecreational\tdrugs\nEvaluate\n\t\nyour\n\t\nstress\n\t\nlevel\n\t\nas\n\t\nfollows:\nNumber\tof\tItems\tChecked\t\nStress\tLevel\n0–7\t\nLow\n8–14\t\nModerate\n15–21\t\nHigh\n22+\t\nVery\n\t\nHigh\n24\tPositive\tCoping\tStrategies\tfor\tStress'),
 Document(id='2b165c65-b341-4079-94b8-f244efd2089c', metadata={'page': 335.0, 'source': 'Data\\book.pdf'}, page_content='Temper\tflare-ups\nStomach\tpain\tor\tulcer\nCrying\tspells\nConstipation\nNightmares\nHypoglycemia\nApathy\nAppetite\tchange\nSexual\tproblems\nColds\nWeight\tchange\nProfuse\tperspiration\nOvereating

In [81]:
# from langchain_openai import OpenAI
# llm = OpenAI(temperature=0.4,max_tokens=500)

In [84]:
pip install langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [86]:
# Import necessary libraries
from langchain_groq import ChatGroq  # Correct import for Groq integration

# Instantiate the Groq model
llm = ChatGroq(
    model="mixtral-8x7b-32768",  # Specify your desired model here
    temperature=0.4,
    max_tokens=500,
    max_retries=2  # Optional parameter for retries
)

# # Use the model to invoke a query
# response = llm.invoke({"input": "What is stress?"})
# print(response["answer"])

In [87]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrived context to answer."
    "the question , If you don't know the answer , say that you"
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human","{input}"),
    ]
)

In [88]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriver,question_answer_chain)

In [91]:
response = rag_chain.invoke({"input": "how to manage time?"})
print(response["answer"])

Developing good time management skills may require giving up some responsibilities. Time management involves organizing daily activities over time, and effective time management can lead to more downtime for rest, recreation, and relationships.
